- [ ] Ler todos os arquivos
- [ ] Separar por municipios
- [ ] Aplicar funcao de limpeza
- [ ] Salvar no banco

# Funçao de limpeza

In [ ]:
def suavizar(df, window_size=3, threshold=2):
    """
    Smooth data by replacing outliers with previous day's casosNovos.
    
    Parameters:
    - df: pandas DataFrame with columns 'date' and 'casosNovos'
    - window_size: number of days to consider in rolling window
    - threshold: number of standard deviations to use for outlier detection
    
    Returns:
    - DataFrame with smoothed values
    """
    # Make a copy to avoid modifying original data
    df_smoothed = df.copy()
    
    # Calculate rolling statistics
    rolling_mean = df['casosNovos'].rolling(window=window_size, center=True, min_periods=1).mean()
    rolling_std = df['casosNovos'].rolling(window=window_size, center=True, min_periods=1).std()
    
    # Identify outliers (values outside mean ± threshold*std)
    lower_bound = rolling_mean - threshold * rolling_std
    upper_bound = rolling_mean + threshold * rolling_std
    
    is_outlier = (df['casosNovos'] < lower_bound) | (df['casosNovos'] > upper_bound)
    
    # Replace outliers with previous day's value
    df_smoothed['casosNovos'] = df['casosNovos'].where(~is_outlier, df['casosNovos'].shift(1))
    
    # For the first row (no previous value), use the next value if available
    if is_outlier.iloc[0] and len(df) > 1:
        df_smoothed.iloc[0, df_smoothed.columns.get_loc('casosNovos')] = df['casosNovos'].iloc[1]
    
    return df_smoothed

# Recalcula casos acumulados

In [ ]:
def recalcula_casos_acumulados(df):
  df['novos_casos_acumulados'] = df['casosNovos'].cumsum()
  return df

# Função de limpeza

In [ ]:
def limpar(df):
  df = suavizar(df)
  df = recalcula_casos_acumulados(df)
  return df